In [42]:
!pip install sqlalchemy


In [62]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table,select,func
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker

In [44]:
### useful: functions for displaying results from sql queries using pandas
from IPython.display import display
import pandas as pd

def sql(query):
    print()
    print(query)
    print()

def get_results(query):
    global engine
    q = query.statement if isinstance(query, sqlalchemy.orm.query.Query) else query
    return pd.read_sql(q, engine)

def display_results(query):
    df = get_results(query)
    display(df)
    sql(query)


In [45]:
#Exo1
engine = create_engine("sqlite:///chinook.db")

# Connect to the database
cur = engine.connect()
### useful: extract classes from the chinook database
metadata = sqlalchemy.MetaData()
metadata.reflect(engine)

## we need to do this once
from sqlalchemy.ext.automap import automap_base

# produce a set of mappings from this MetaData.
Base = automap_base(metadata=metadata)

# calling prepare() just sets up mapped classes and relationships.
Base.prepare()

# also prepare an orm session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [46]:
#Exo2
metadata = MetaData()
metadata.reflect(bind=engine)
print(metadata.tables.keys())


dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'tracks', 'media_types', 'invoices', 'playlist_track', 'playlists'])


In [47]:
#Exo3
track_table=metadata.tables['tracks']
query=select(track_table).limit(3)
sql(query)
get_results(query)
display_results(query)




SELECT tracks."TrackId", tracks."Name", tracks."AlbumId", tracks."MediaTypeId", tracks."GenreId", tracks."Composer", tracks."Milliseconds", tracks."Bytes", tracks."UnitPrice" 
FROM tracks
 LIMIT :param_1



,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99



SELECT tracks."TrackId", tracks."Name", tracks."AlbumId", tracks."MediaTypeId", tracks."GenreId", tracks."Composer", tracks."Milliseconds", tracks."Bytes", tracks."UnitPrice" 
FROM tracks
 LIMIT :param_1



In [59]:
track_table=metadata.tables['invoices']
print(track_table.columns.keys())



['InvoiceId', 'CustomerId', 'InvoiceDate', 'BillingAddress', 'BillingCity', 'BillingState', 'BillingCountry', 'BillingPostalCode', 'Total']


In [53]:
#Exo4
album_table=metadata.tables['albums']
query=select(track_table.columns.Name,album_table.columns.Title).join(album_table,track_table.columns.AlbumId==album_table.columns.AlbumId).limit(20)
sql(query)
get_results(query)
display_results(query)



SELECT tracks."Name", albums."Title" 
FROM tracks JOIN albums ON tracks."AlbumId" = albums."AlbumId"
 LIMIT :param_1



,Name,Title
0,For Those About To Rock (We Salute You),For Those About To Rock We Salute You
1,Put The Finger On You,For Those About To Rock We Salute You
2,Let's Get It Up,For Those About To Rock We Salute You
3,Inject The Venom,For Those About To Rock We Salute You
4,Snowballed,For Those About To Rock We Salute You
5,Evil Walks,For Those About To Rock We Salute You
6,C.O.D.,For Those About To Rock We Salute You
7,Breaking The Rules,For Those About To Rock We Salute You
8,Night Of The Long Knives,For Those About To Rock We Salute You
9,Spellbound,For Those About To Rock We Salute You



SELECT tracks."Name", albums."Title" 
FROM tracks JOIN albums ON tracks."AlbumId" = albums."AlbumId"
 LIMIT :param_1



In [65]:
#Exo5
#print out the first 10 track sales from the invoice_items table
#for these first 10 sales, print what are the names of the track sold, and the quantity sold
invoice_items_table=metadata.tables['invoice_items']
track_table=metadata.tables['tracks']
query=select(func.sum(invoice_items_table.columns.Quantity),track_table.columns.Name).join(track_table,invoice_items_table.columns.TrackId==track_table.columns.TrackId).group_by(track_table.columns.Name).limit(10)
sql(query)
get_results(query)
display_results(query)



SELECT sum(invoice_items."Quantity") AS sum_1, tracks."Name" 
FROM invoice_items JOIN tracks ON invoice_items."TrackId" = tracks."TrackId" GROUP BY tracks."Name"
 LIMIT :param_1



,sum_1,Name
0,1,"""?"""
1,1,#9 Dream
2,1,'Round Midnight
3,1,(Anesthesia) Pulling Teeth
4,1,(White Man) In Hammersmith Palais
5,1,(Wish I Could) Hideaway
6,1,...And Found
7,1,...And Justice For All
8,1,01 - Prowler
9,1,04 - Running Free



SELECT sum(invoice_items."Quantity") AS sum_1, tracks."Name" 
FROM invoice_items JOIN tracks ON invoice_items."TrackId" = tracks."TrackId" GROUP BY tracks."Name"
 LIMIT :param_1



In [71]:
#Exo6
#print the names of top 10 tracks sold, and how many they times they were sold
invoice_items_table = metadata.tables['invoice_items']
track_table = metadata.tables['tracks']

query = (select(
    track_table.c.Name.label('TrackName'),
    func.sum(invoice_items_table.c.Quantity).label('TimesSold')
)
.join(track_table, invoice_items_table.c.TrackId == track_table.c.TrackId)
.group_by(track_table.c.TrackId, track_table.c.Name)
.order_by(func.sum(invoice_items_table.c.Quantity).desc())
.limit(10))

display_results(query)

,TrackName,TimesSold
0,Balls to the Wall,2
1,Inject The Venom,2
2,Snowballed,2
3,Overdose,2
4,Deuces Are Wild,2
5,Not The Doctor,2
6,Por Causa De Você,2
7,Welcome Home (Sanitarium),2
8,Snowblind,2
9,Cornucopia,2



SELECT tracks."Name" AS "TrackName", sum(invoice_items."Quantity") AS "TimesSold" 
FROM invoice_items JOIN tracks ON invoice_items."TrackId" = tracks."TrackId" GROUP BY tracks."TrackId", tracks."Name" ORDER BY sum(invoice_items."Quantity") DESC
 LIMIT :param_1



In [72]:
print(metadata.tables.keys())


dict_keys(['albums', 'artists', 'customers', 'employees', 'genres', 'invoice_items', 'tracks', 'media_types', 'invoices', 'playlist_track', 'playlists'])


In [80]:
invoice_items_table = metadata.tables['invoice_items']
print(invoice_items_table.columns.keys())
artists_table = metadata.tables['artists']
print(artists_table.columns.keys())
tracks_table = metadata.tables['tracks']
print(tracks_table.columns.keys())
albums_table = metadata.tables['albums']
print(albums_table.columns.keys())

['InvoiceLineId', 'InvoiceId', 'TrackId', 'UnitPrice', 'Quantity']
['ArtistId', 'Name']
['TrackId', 'Name', 'AlbumId', 'MediaTypeId', 'GenreId', 'Composer', 'Milliseconds', 'Bytes', 'UnitPrice']
['AlbumId', 'Title', 'ArtistId']


In [81]:
#Exo7
#Who are the top 10 highest selling artists?
invoice_items_table = metadata.tables['invoice_items']
artists_table = metadata.tables['artists']
tracks_table = metadata.tables['tracks']
albums_table = metadata.tables['albums']
query=select(artists_table.columns.Name).join(albums_table,artists_table.columns.ArtistId==albums_table.columns.ArtistId).join(tracks_table,albums_table.columns.AlbumId==tracks_table.columns.AlbumId).join(invoice_items_table,tracks_table.columns.TrackId==invoice_items_table.columns.TrackId).group_by(artists_table.columns.Name).order_by(func.sum(invoice_items_table.columns.Quantity).desc()).limit(10)
sql(query)
get_results(query)
display_results(query)


SELECT artists."Name" 
FROM artists JOIN albums ON artists."ArtistId" = albums."ArtistId" JOIN tracks ON albums."AlbumId" = tracks."AlbumId" JOIN invoice_items ON tracks."TrackId" = invoice_items."TrackId" GROUP BY artists."Name" ORDER BY sum(invoice_items."Quantity") DESC
 LIMIT :param_1



,Name
0,Iron Maiden
1,U2
2,Metallica
3,Led Zeppelin
4,Os Paralamas Do Sucesso
5,Deep Purple
6,Faith No More
7,Lost
8,Eric Clapton
9,R.E.M.



SELECT artists."Name" 
FROM artists JOIN albums ON artists."ArtistId" = albums."ArtistId" JOIN tracks ON albums."AlbumId" = tracks."AlbumId" JOIN invoice_items ON tracks."TrackId" = invoice_items."TrackId" GROUP BY artists."Name" ORDER BY sum(invoice_items."Quantity") DESC
 LIMIT :param_1

